In [38]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.express as px
import plotly.figure_factory as ff
import json
import geojson
import requests
import plotly.graph_objs as go
import plotly.graph_objects as go

from plotly.subplots import make_subplots

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

#%matplotlib notebook
pd.options.display.max_columns = 150
pd.options.display.max_rows = 2000

data_root = '/Users/charlessimpson/Library/Mobile Documents/com~apple~CloudDocs/Documents/WorkspacePython/DATA/SQL/'

In [39]:
df2000=pd.read_csv("../csv/deces-2000.csv", sep=';', encoding='utf-8')
df2001=pd.read_csv("../csv/deces-2001.csv", sep=';', encoding='utf-8')
df2002=pd.read_csv("../csv/deces-2002.csv", sep=';', encoding='utf-8')
df2003=pd.read_csv("../csv/deces-2003.csv", sep=';', encoding='utf-8')
df2004=pd.read_csv("../csv/deces-2004.csv", sep=';', encoding='utf-8')
df2005=pd.read_csv("../csv/deces-2005.csv", sep=';', encoding='utf-8')
df2006=pd.read_csv("../csv/deces-2006.csv", sep=';', encoding='utf-8')
df2007=pd.read_csv("../csv/deces-2007.csv", sep=';', encoding='utf-8')
df2008=pd.read_csv("../csv/deces-2008.csv", sep=';', encoding='utf-8')
df2009=pd.read_csv("../csv/deces-2009.csv", sep=';', encoding='utf-8')
df2010=pd.read_csv("../csv/deces-2010.csv", sep=';', encoding='utf-8')
df2011=pd.read_csv("../csv/deces-2011.csv", sep=';', encoding='utf-8')
df2012=pd.read_csv("../csv/deces-2012.csv", sep=';', encoding='utf-8')
df2013=pd.read_csv("../csv/deces-2013.csv", sep=';', encoding='utf-8')
df2014=pd.read_csv("../csv/deces-2014.csv", sep=';', encoding='utf-8')
df2015=pd.read_csv("../csv/deces-2015.csv", sep=';', encoding='utf-8')
df2016=pd.read_csv("../csv/deces-2016.csv", sep=';', encoding='utf-8')
df2017=pd.read_csv("../csv/deces-2017.csv", sep=';', encoding='utf-8')
df2018=pd.read_csv("../csv/deces-2018.csv", sep=';', encoding='utf-8')
df2019=pd.read_csv("../csv/deces-2019.csv", sep=';', encoding='utf-8')
df2020 = pd.read_csv('../csv/deces_2020.csv', sep=';', encoding='utf-8')
df_01_2021 = pd.read_csv('../csv/Deces_2021_M01.csv', sep=';', encoding='utf-8')
df_02_2021 = pd.read_csv('../csv/Deces_2021_M02.csv', sep=';', encoding='utf-8')
df_03_2021 = pd.read_csv('../csv/Deces_2021_M03.csv', sep=';', encoding='utf-8')
df_04_2021 = pd.read_csv('../csv/Deces_2021_M04.csv', sep=';', encoding='utf-8')
df_05_2021 = pd.read_csv('../csv/Deces_2021_M05.csv', sep=';', encoding='utf-8')
df_06_2021 = pd.read_csv('../csv/Deces_2021_M06.csv', sep=';', encoding='utf-8')
df_07_2021 = pd.read_csv('../csv/Deces_2021_M07.csv', sep=';', encoding='utf-8')
df_08_2021 = pd.read_csv('../csv/Deces_2021_M08.csv', sep=';', encoding='utf-8')
df_09_2021 = pd.read_csv('../csv/Deces_2021_M09.csv', sep=';', encoding='utf-8')
df_10_2021 = pd.read_csv('../csv/Deces_2021_M10.csv', sep=';', encoding='utf-8')
df_11_2021 = pd.read_csv('../csv/Deces_2021_M11.csv', sep=';', encoding='utf-8')
df_12_2021 = pd.read_csv('../csv/Deces_2021_M12.csv', sep=';', encoding='utf-8')

# Je concat toutes les BDD
data = pd.concat([df2000,df2001,df2002,df2003,df2004,df2005,df2006,df2007,df2008,df2009,df2010,df2011,df2012,df2013,df2014,df2015,df2016,df2017,df2018,df2019,df2020,df_01_2021,df_02_2021,df_03_2021,df_04_2021,df_05_2021,
                  df_06_2021,df_07_2021,df_08_2021,df_09_2021,df_10_2021,df_11_2021,df_12_2021])

# je récupère le numéro du département à 2 chiffres (attention aux 10 premiers départements)
def get_department(postal_code):
    postal_code = str(postal_code)
    return postal_code[:2]

# Renomage des colonnes
db = data.rename(columns={'nomprenom':'nom',
                        'datenaiss':'date_naiss',
                        'lieunaiss':'dpt_lieu_naiss',
                        'sexe':'sexe',
                        'commnaiss':'ville',
                        'paysnaiss':'pays',
                        'datedeces':'date_dc',
                        'lieudeces':'dpt_lieu_dc',
                        'actedeces':'dpt_lieu_deces'})

# je réduis ma BDD à la taille utile
#db = db[['nom','dpt_lieu_dc', 'date_naiss','date_dc']]

# je récupère les numéros de départements simplifiés pour les décès
db['dpt_deces'] = db['dpt_lieu_dc'].apply(lambda x : get_department(x))
print("Nombre de deces cumulés sur la période",db.shape)

# Nettoyage des numéros de départements pour s'assurer qu'on récupère correctement les 10 premiers départements
def non_int_filter(dpt_deces):
    try:
        return int(dpt_deces)
    except:
        return 0
db['dpt_deces'] = db['dpt_deces'].apply(lambda x : non_int_filter(x))
count_db = db.loc[db['dpt_deces']== 0]


# je récupère les numéros de départements simplifiés pour les naissance
db['dpt_lieu_naiss'] = db['dpt_lieu_naiss'].apply(lambda x : get_department(x))
db['dpt_lieu_naiss'] = db['dpt_lieu_naiss'].apply(lambda x : non_int_filter(x))
count_db = db.loc[db['dpt_lieu_naiss']== 0]


# J'identifie les décès sans département
db = db.loc[db['dpt_deces']!= 0] # fonction qui supprimer les lignes sans départements
# transformer la data décès au bon format pour manipuler cette donnée 
db['date_deces']=pd.to_datetime((db.date_dc), format='%Y%m%d', errors='coerce')

# Nettoyage du format de la date de décès, pour avoir le mois et l'année au bon format
db=db.dropna(subset=['date_deces'])

def year(date_dc):
    date_dc = str(date_dc)
    return date_dc[:4]
db['year'] = db['date_dc'].apply(lambda x : year(x))

def month(date_dc):
    date_dc = str(date_dc)
    return date_dc[:6]
db['month'] = db['date_dc'].apply(lambda x : month(x))

#Suppression des DOMTOM
db=db[db['dpt_deces']!=97]
db=db[db['dpt_deces']!=98]
db=db[db['dpt_deces']!=99]

# Je rajoute la BDD des départements pour insérer les noms des départements
departements_info = pd.read_excel('../csv/departements-francais.xls')
number_to_departement = departements_info.set_index('NUMÉRO')['NOM'].to_dict()

# Nettoyage des déparmtents, j'enlève l'outre mer et je rajouter le nom des départements
number_to_departement[97] =  'Outre-mer'
db = db.rename(columns={'dpt_deces':'departement_number'})
db['departement'] = db['departement_number'].apply(lambda x : number_to_departement.get(x, ""))
wrong_dpt = db[db.departement_number.isin([98, 99])]
db = db[~db.departement_number.isin([98, 99])]

# je modifie les noms des départements qui ne sont pas correctes
db['departement'] = db['departement'].str.replace("Val-D'Oise", "Val-d'Oise")
db['departement'] = db['departement'].str.replace( 'Seine-St-Denis', "Seine-Saint-Denis")
db['departement'] = db['departement'].str.replace( "Côtes d'Armor", "Côtes-d'Armor")
db['departement'] = db['departement'].str.replace( "Vandée", "Vendée")


db.sample(3)

Nombre de deces cumulés sur la période (12792304, 10)


,nom,sexe,date_naiss,dpt_lieu_naiss,ville,pays,date_dc,dpt_lieu_dc,dpt_lieu_deces,departement_number,date_deces,year,month,departement
401467,BENKEDDACHE*BRIGITTE/,2,19590805,19,USSEL,NaN,20120907.0,23144,6,23,2012-09-07,2012,201209,Creuse
57113,HENNUY*SERGE HENRI ROBERT/,1,19481228,8,MOHON,NaN,20170128.0,85191,148,85,2017-01-28,2017,201701,Vendée
18950,RAYMOND*JAMES/,1,19290206,17,AGUDELLE,NaN,20161230.0,33243,4,33,2016-12-30,2016,201612,Gironde


In [40]:
db.year.dtype

dtype('O')

In [41]:
# création du DATA qui permet de générer les différents graphiques. nous nous intéressons à 2000, 2010, 2020 et 2021. Att il y a des chiffres entre ces années qui se remontés par le skype 
# ne pas y faire attention


db=db[db['year']>'1999']
db_year=db.pivot_table(index='departement',columns='year', values='nom', aggfunc='count').fillna(0).reset_index()
#db_year.loc['total']=db_year.sum(axis=0)
db_year['variation entre 2021 et 2020'] = round((db_year['2021'] - db_year['2020']) / db_year['2020']* 100,1)
db_year['variation entre 2021 et 2010'] = round((db_year['2021'] - db_year['2010']) / db_year['2020']* 100,1)
db_year['variation entre 2021 et 2000'] = round((db_year['2021'] - db_year['2000']) / db_year['2020']* 100,1)
db_year=db_year[['departement','2000','2010','2020','2021','variation entre 2021 et 2020','variation entre 2021 et 2010','variation entre 2021 et 2000']]
db_year.tail(5)# DATA décès par année# DATA décès par année

year,departement,2000,2010,2020,2021,variation entre 2021 et 2020,variation entre 2021 et 2010,variation entre 2021 et 2000
90,Vendée,5213.0,5757.0,7065.0,7344.0,3.9,22.5,30.2
91,Vienne,4245.0,4407.0,5043.0,5038.0,-0.1,12.5,15.7
92,Vosges,3841.0,3746.0,4777.0,4387.0,-8.2,13.4,11.4
93,Yonne,3925.0,3798.0,4367.0,4253.0,-2.6,10.4,7.5
94,Yvelines,8054.0,7960.0,9953.0,8733.0,-12.3,7.8,6.8


In [42]:
# Fichier à télécharger ici : https://github.com/gregoiredavid/france-geojson/blob/master/departements.geojson

with open ("../geojson/departements.geojson", "r") as f:
    departements = geojson.load(f)
    
with open ("../geojson/villes_principales_france.geojson", "r") as f2:
    villes_principales = geojson.load(f2)
    
villes_principales = villes_principales.features

with open ("../geojson/idf.geojson", "r") as f3:
    idf = geojson.load(f3)

for i in range(1, len(db_year.columns)):
    print(db_year.columns[i])

def generateMaps(columns, fileName, description, title, geojson):
    # on génère les cartes en fonction des colonnes présentes
    for i in range(1, len(columns)):
        
        column = columns[i]
        dataSet = db_year

        # on récupère les départements non présents
        deps=[]
        for i, d in dataSet.iterrows():
            deps.append(d['departement'])
        for dpt in departements.features:
            if dpt["properties"]["nom"] not in deps:
                dataSet = dataSet.append({"departement": dpt["properties"]["nom"], column: 0}, ignore_index = True)
        
        # si la carte qu'on cherche à générer est celle de l'IDF, on modifie le dataFrame
        if 'idf' in fileName:
            baseDataSet = dataSet
            dataSet = pd.DataFrame([], columns=['departement', column])
            for i, d in baseDataSet.iterrows():
                x = d['departement']
                if x == 'Paris' or x == 'Val-de-Marne' or x == 'Val-d\'Oise' or x == 'Seine-Saint-Denis' or x == 'Essonne' or x == 'Hauts-de-Seine' or x == 'Yvelines' or x == 'Seine-et-Marne':
                    dataSet.loc[i] = [x, d[column]]
        
        prefix = ''
        # si la colonne est un agrégat comme "50y", on ajoute un préfixe + " sur 50 ans" au titre de la carte (fonctionne avec n'importe quel nombre)
        columnDesc = ''
        if column[-1] == 'y' and column[len(column) - 2].isdigit() and column[len(column) - 3].isdigit():
            columnDesc = ' sur ' + column[0] + column[1] + ' ans'
            prefix = column + '-'
        
        # si la colonne est une année comme 2020, on ajoute un préfixe + " en 2020" au titre de la carte
        if len(column) == 4 and column.isdigit():
            columnDesc = ' en ' + column
            prefix = column + '-'
        
        # on définit la fig de la carte à générer
        fig = px.choropleth(
            dataSet,
            geojson=geojson,
            locations='departement',
            scope='europe',
            color=dataSet[column],
            range_color=(dataSet[column].min(), dataSet[column].max()),
            featureidkey="properties.nom",
            hover_data=[column,'variation entre 2021 et 2020','variation entre 2021 et 2010','variation entre 2021 et 2000'],
            # change
            labels={column: description + columnDesc, 'departement': 'Département'}
        )

        # on affiche les villes principales sur la fig avec le geojson chargé précédemment
        for ville in villes_principales:
            # if carte IDF paris 
            if 'idf' in fileName:
                if ville.properties['nom'] == 'Paris':
                    fig.add_trace(go.Scattergeo(
                    lon = [ville.geometry.coordinates[0]],
                    lat = [ville.geometry.coordinates[1]],
                    text = ville.properties['nom'],
                    textfont=dict(
                        size=12,
                        color="white"
                    ),
                    mode="text",
                    showlegend = False,
                    hoverinfo="skip"
                ))
            else:
                fig.add_trace(go.Scattergeo(
                    lon = [ville.geometry.coordinates[0]],
                    lat = [ville.geometry.coordinates[1]],
                    text = ville.properties['nom'],
                    textfont=dict(
                        size=12,
                        color="white"
                    ),
                    mode="text",
                    showlegend = False,
                    hoverinfo="skip"
                ))

        if 'idf' in fileName:
            fig.update_layout(title_text = "")
            fig.update_layout(margin=dict(l=40, r=40, t=40, b=40), showlegend=False, width=400, height=400)
            fig.update_coloraxes(showscale=False)
        else:
            fig.update_layout(title_text = title + columnDesc)
            fig.update_layout(margin=dict(l=40, r=40, t=40, b=40))   
        
        fig.update_geos(fitbounds="locations", visible=False)

        # génération du fichier de la carte donnée
        fig.write_html("cartes-inter/" + prefix + fileName + ".html")
        #fig.show()

generateMaps(db_year.columns, "deces-departements", "Nombre de décès", "Nombre de décès par département", departements)
# le fichier cartes-inter/concessions-departements-yc-idf.html peut désormais être rafraîchi puisqu'il dépend des cartes qu'on génère ci-dessus

generateMaps(db_year.columns, "deces-idf", "Nombre de décès", "Nombre de décès par département en Ile-de-France", idf)# Analyse des décès par âge moyen# DATA décès par année

2000
2010
2020
2021
variation entre 2021 et 2020
variation entre 2021 et 2010
variation entre 2021 et 2000


ValueError: Value of 'hover_data_1' is not the name of a column in 'data_frame'. Expected one of ['departement', '2000'] but received: variation entre 2021 et 2020